In [1]:
import pandas as pd
import numpy as np
import time
import datetime

%run "../1. Librerias Mongo/MongoDB_Connections.ipynb"
%run "../1. Librerias Mongo/MongoDB_Funciones_Consultas.ipynb"

### Carga de datos de entrada

In [2]:
# MOVIMIENTOS

#DF = pd.read_csv('../../data/DataFrame_Final_Cierre.csv', parse_dates=['FECHA'])
df_tracks = pd.read_csv('../../data/DataFrame_Final_Cierre_Cluster_2017_2019.csv', parse_dates=['FECHA'])

df_clusterFull = pd.read_csv('../../data/Clusterizada_DataFull.csv')
df_clusterGeog5 = pd.read_csv('../../data/Clusterizada_soloGeografico_5Cluster.csv')
df_clusterGeog7 = pd.read_csv('../../data/Clusterizada_soloGeografico_7Cluster.csv')
df_clusterDmnd = pd.read_csv('../../data/Clusterizada_soloDemandaMensual.csv')


In [3]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2912138 entries, 0 to 2912137
Data columns (total 25 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ESTACION             int64         
 1   ANIO                 int64         
 2   MES                  int64         
 3   DIA                  int64         
 4   HORA                 int64         
 5   FECHA                datetime64[ns]
 6   DIA_SEMANA           int64         
 7   AM_PM                object        
 8   TEMPORADA            object        
 9   TEMPORADA_NUM        int64         
 10  Es_Festivo           int64         
 11  Es_FinSemana         int64         
 12  TEMPERATURA          float64       
 13  VIENTO               float64       
 14  PRESION              float64       
 15  HUMEDAD              float64       
 16  PRECIPITACION_1h     float64       
 17  PRECIPITACION_3h     float64       
 18  DESC_TIEMPO          object        
 19  DESC_TIEMPO_detalle  

In [4]:
# ESTACIONES: Lista de estaciones en DF

df_estaciones = df_tracks.ESTACION.unique()
df_estaciones.sort()
df_estaciones

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  69,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,
        82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,
        95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107,
       108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
       121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
       134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
       147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,
       160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
       173, 174, 175], dtype=int64)

In [5]:
# Meteorologia

db_Meteo = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 'Meteo_US_NivelHora')

t_ini = time.time()
#data_Meteo = _consulta_meteoUS_por_anio(db_Meteo, 2020)
data_Meteo = _consulta_meteoUS_full(db_Meteo)
t_end = time.time()

print (t_end - t_ini)

# Se agrega campo FECHA para join
data_Meteo['FECHA'] =pd.to_datetime({'year': data_Meteo['ANIO'],
                                          'month': data_Meteo["MES"]
                                          ,'day':  data_Meteo['DIA']
                                          ,'hour':data_Meteo['HORA']},
                                          format='%d-%m-%Y', errors='coerce')

data_Meteo.info()

Conexion OK
Collection(Database(MongoClient(host=['ac-x1d17w2-shard-00-02.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-01.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-00.15npsxw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-xyv6ql-shard-0', tls=True, serverselectiontimeoutms=4000, tlscafile='C:\\ProgramData\\Anaconda3\\lib\\site-packages\\certifi\\cacert.pem'), 'BiciMAD'), 'Meteo_US_NivelHora')
1.7830934524536133
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49368 entries, 0 to 49367
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ANIO                 49368 non-null  int64         
 1   MES                  49368 non-null  int64         
 2   DIA                  49368 non-null  int64         
 3   HORA                 49368 non-null  int64         
 4   TEMPERATURA          49368 non-null  float64       
 5 

In [6]:
data_Meteo

,ANIO,MES,DIA,HORA,TEMPERATURA,TEMP_SENS_TERMC,VIENTO,PRESION,HUMEDAD,PRECIPITACION_1h,PRECIPITACION_3h,NIEVE_1h,NIEVE_3h,ID_TIPO_TIEMPO,DESC_TIEMPO,DESC_TIEMPO_detalle,FECHA
0,2017,1,1,0,-3.42,-5.73,1.54,1030.0,93.0,0.0,0.0,0.0,0.0,800,Clear,sky is clear,2017-01-01 00:00:00
1,2017,1,1,1,-4.42,-4.42,1.03,1029.0,93.0,0.0,0.0,0.0,0.0,800,Clear,sky is clear,2017-01-01 01:00:00
2,2017,1,1,2,-4.42,-4.42,0.51,1028.0,93.0,0.0,0.0,0.0,0.0,802,Clouds,scattered clouds,2017-01-01 02:00:00
3,2017,1,1,3,-3.42,-3.42,1.03,1028.0,86.0,0.0,0.0,0.0,0.0,803,Clouds,broken clouds,2017-01-01 03:00:00
4,2017,1,1,4,-2.42,-6.50,3.10,1028.0,86.0,0.0,0.0,0.0,0.0,803,Clouds,broken clouds,2017-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49363,2022,8,19,19,29.39,27.85,1.54,1019.0,23.0,0.0,0.0,0.0,0.0,800,Clear,sky is clear,2022-08-19 19:00:00
49364,2022,8,19,20,27.07,26.39,1.03,1006.0,28.0,0.0,0.0,0.0,0.0,800,Clear,sky is clear,2022-08-19 20:00:00
49365,2022,8,19,21,26.27,26.27,1.03,1006.0,28.0,0.0,0.0,0.0,0.0,800,Clear,sky is clear,2022-08-19 21:00:00
49366,2022,8,19,22,25.11,24.46,3.09,1006.0,30.0,0.0,0.0,0.0,0.0,800,Clear,sky is clear,2022-08-19 22:00:00


### Creación de DF de fechas

In [7]:
# FECHAS DE FESTIVOS

df_festivos_in = pd.read_csv('../../data/Festivos_Madrid.csv', sep=';')
# Se agrega campo FECHA para join
df_festivos_in['FECHA'] =pd.to_datetime({'year': df_festivos_in['ANIO'],
                                          'month': df_festivos_in["MES"]
                                          ,'day':  df_festivos_in['DIA']},
                                          format='%d-%m-%Y', errors='coerce')
df_festivos_in.info()
df_festivos = df_festivos_in[df_festivos_in['FESTIVO']==1]

df_festivos.info()
df_festivos[df_festivos['ANIO']==2018].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ANIO     1461 non-null   int64         
 1   MES      1461 non-null   int64         
 2   DIA      1461 non-null   int64         
 3   FESTIVO  1461 non-null   int64         
 4   FECHA    1461 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(4)
memory usage: 57.2 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 0 to 1454
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ANIO     57 non-null     int64         
 1   MES      57 non-null     int64         
 2   DIA      57 non-null     int64         
 3   FESTIVO  57 non-null     int64         
 4   FECHA    57 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(4)
memory usage: 2.7 KB


,ANIO,MES,DIA,FESTIVO,FECHA
365,2018,1,1,1,2018-01-01
370,2018,1,6,1,2018-01-06
452,2018,3,29,1,2018-03-29
453,2018,3,30,1,2018-03-30
485,2018,5,1,1,2018-05-01


### Creación de rango total de fechas

In [8]:
# Creación de rangos de fecha válidos
# sdate = datetime.strptime('01/01/2018 00', '%d/%m/%Y %H')
# edate = datetime.strptime('31/01/2018 23', '%d/%m/%Y %H')

sdate = np.datetime64('2018-01-01T00:00:00')
edate = np.datetime64('2020-02-29T23:00:00')

df_dates=pd.date_range(start=sdate, end=edate, freq='H')

print(df_dates.shape)
df_dates

(18960,)


DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 01:00:00',
               '2018-01-01 02:00:00', '2018-01-01 03:00:00',
               '2018-01-01 04:00:00', '2018-01-01 05:00:00',
               '2018-01-01 06:00:00', '2018-01-01 07:00:00',
               '2018-01-01 08:00:00', '2018-01-01 09:00:00',
               ...
               '2020-02-29 14:00:00', '2020-02-29 15:00:00',
               '2020-02-29 16:00:00', '2020-02-29 17:00:00',
               '2020-02-29 18:00:00', '2020-02-29 19:00:00',
               '2020-02-29 20:00:00', '2020-02-29 21:00:00',
               '2020-02-29 22:00:00', '2020-02-29 23:00:00'],
              dtype='datetime64[ns]', length=18960, freq='H')

### EJECUCION DE PROCESO 

In [9]:
# df_estaciones = [175]
# type(df_estaciones)
df_data = df_tracks[df_tracks['ESTACION']==175]
df_data

,ESTACION,ANIO,MES,DIA,HORA,FECHA,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,...,HUMEDAD,PRECIPITACION_1h,PRECIPITACION_3h,DESC_TIEMPO,DESC_TIEMPO_detalle,CLUSTER_FullCols,CLUSTER_soloGeo5,CLUSTER_soloGeo7,CLUSTER_soloDemanda,DEMANDA
137823,175,2017,5,22,11,2017-05-22 11:00:00,2,AM,PRIMAVERA,2,...,32.0,0.0,0.0,Clear,sky is clear,0,0,1,2,5
137982,175,2017,5,22,12,2017-05-22 12:00:00,2,PM,PRIMAVERA,2,...,29.0,0.0,0.0,Clear,sky is clear,0,0,1,2,1
138147,175,2017,5,22,13,2017-05-22 13:00:00,2,PM,PRIMAVERA,2,...,26.0,0.0,0.0,Clear,sky is clear,0,0,1,2,1
138309,175,2017,5,22,14,2017-05-22 14:00:00,2,PM,PRIMAVERA,2,...,24.0,0.0,0.0,Clear,sky is clear,0,0,1,2,3
138473,175,2017,5,22,15,2017-05-22 15:00:00,2,PM,PRIMAVERA,2,...,21.0,0.0,0.0,Clear,sky is clear,0,0,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911680,175,2020,2,29,19,2020-02-29 19:00:00,7,PM,INVIERNO,1,...,60.0,0.0,0.0,Clouds,few clouds,0,0,1,2,8
2911809,175,2020,2,29,20,2020-02-29 20:00:00,7,PM,INVIERNO,1,...,61.0,0.0,0.0,Clouds,few clouds,0,0,1,2,3
2911921,175,2020,2,29,21,2020-02-29 21:00:00,7,PM,INVIERNO,1,...,67.0,0.0,0.0,Clouds,few clouds,0,0,1,2,1
2912014,175,2020,2,29,22,2020-02-29 22:00:00,7,PM,INVIERNO,1,...,69.0,0.0,0.0,Clouds,few clouds,0,0,1,2,2


In [10]:
clusterFull = df_clusterFull.loc[df_clusterFull['ESTACION'] == df_estaciones[0], ['CLUSTER']].iloc[0]['CLUSTER']
type(clusterFull)

numpy.int64

In [11]:
i=0
df_final = pd.DataFrame(columns = ['ESTACION', 'ANIO', 'MES', 'DIA', 'HORA', 'FECHA', 'DIA_SEMANA', 'AM_PM', 'TEMPORADA', 
                            'TEMPORADA_NUM', 'Es_Festivo', 'Es_FinSemana', 'TEMPERATURA', 'VIENTO', 'PRESION', 'HUMEDAD', 
                            'PRECIPITACION_1h', 'PRECIPITACION_3h', 'DESC_TIEMPO', 'DESC_TIEMPO_detalle', 'CLUSTER_FullCols', 
                            'CLUSTER_soloGeo5', 'CLUSTER_soloGeo7', 'CLUSTER_soloDemanda', 'DEMANDA'])

t_ini = time.time()

for estacion in df_estaciones:
    print('Estacion: '+str(estacion))
    
    df_data = df_tracks[df_tracks['ESTACION']==estacion]
    
    df_final_estacion = pd.DataFrame(columns = ['ESTACION', 'ANIO', 'MES', 'DIA', 'HORA', 'FECHA', 'DIA_SEMANA', 'AM_PM', 'TEMPORADA', 
                            'TEMPORADA_NUM', 'Es_Festivo', 'Es_FinSemana', 'TEMPERATURA', 'VIENTO', 'PRESION', 'HUMEDAD', 
                            'PRECIPITACION_1h', 'PRECIPITACION_3h', 'DESC_TIEMPO', 'DESC_TIEMPO_detalle', 'CLUSTER_FullCols', 
                            'CLUSTER_soloGeo5', 'CLUSTER_soloGeo7', 'CLUSTER_soloDemanda', 'DEMANDA'])    
    
    t_ini_estacion = time.time()
    for fecha in df_dates:
        
        # Variable dia semana
        if fecha.isoweekday()==7:
            dia_semana=1
        else:
            dia_semana=fecha.isoweekday()+1
            
        # Variable Es_Festivo
        fec_festivo = np.datetime64(fecha.date())        
        if fec_festivo in df_festivos['FECHA'].values:
            es_festivo = 1
        else:
            es_festivo = 0
                        
        # Variable Es_FinSemana
        if ((dia_semana==1) or (dia_semana==7)):
            es_finsemana = 1
        else:
            es_finsemana = 0
        
        # Variable AM_PM
        if fecha.hour<=12:
            ampm='AM'
        else:
            ampm='PM'
            
        # Variable TEMPORADA
        if fecha.month<=3:
            temporada='INVIERNO'
            temporada_num=1
        elif fecha.month<=6:
            temporada='PRIMAVERA'
            temporada_num=2
        elif fecha.month<=9:
            temporada='VERANO'
            temporada_num=3
        else:
            temporada='OTONO'
            temporada_num=4
            
        clusterFull = df_clusterFull.loc[df_clusterFull['ESTACION'] == estacion, ['CLUSTER']].iloc[0]['CLUSTER']        
        clusterGeog5 = df_clusterGeog5.loc[df_clusterGeog5['ESTACION'] == estacion, ['CLUSTER']].iloc[0]['CLUSTER']     
        clusterGeog7 = df_clusterGeog7.loc[df_clusterGeog7['ESTACION'] == estacion, ['CLUSTER']].iloc[0]['CLUSTER']
        clusterDmnd = df_clusterDmnd.loc[df_clusterDmnd['ESTACION'] == estacion, ['CLUSTER']].iloc[0]['CLUSTER']
                
                
        if fecha not in df_data[df_data['ESTACION']==estacion]['FECHA'].values:
            i=i+1
            meteo = data_Meteo[data_Meteo['FECHA']==fecha]
            new_row_dict = {
                'ESTACION': estacion,
                'ANIO': fecha.year,
                'MES': fecha.month,
                'DIA': fecha.day,
                'HORA': fecha.hour,
                'FECHA': fecha,
                'DIA_SEMANA': dia_semana,
                'AM_PM': ampm,
                'TEMPORADA': temporada,
                'TEMPORADA_NUM': temporada_num,
                'Es_Festivo': es_festivo,
                'Es_FinSemana': es_finsemana,
                'TEMPERATURA': float(meteo.TEMPERATURA.iloc[0]),
                'VIENTO': float(meteo.VIENTO.iloc[0]),
                'PRESION': float(meteo.PRESION.iloc[0]),
                'HUMEDAD': float(meteo.HUMEDAD.iloc[0]),
                'PRECIPITACION_1h': float(meteo.PRECIPITACION_1h.iloc[0]),
                'PRECIPITACION_3h': float(meteo.PRECIPITACION_3h.iloc[0]),
                'DESC_TIEMPO': meteo.DESC_TIEMPO.iloc[0],
                'DESC_TIEMPO_detalle': meteo.DESC_TIEMPO_detalle.iloc[0],
                'CLUSTER_FullCols': clusterFull,
                'CLUSTER_soloGeo5': clusterGeog5,
                'CLUSTER_soloGeo7': clusterGeog7,
                'CLUSTER_soloDemanda': clusterDmnd,
                'DEMANDA': 0
            }
            
            new_row = pd.DataFrame(new_row_dict, index=[0])
            
            #print(meteo.TEMPERATURA.iloc[0])
            df_final_estacion = pd.concat([df_final_estacion, new_row])
            
            #df = pd.concat([df, i])
        else:
            #print(df_data[(df_data['ESTACION']==estacion) & (df_data['FECHA']==fecha)])
            old_row = df_data[(df_data['ESTACION']==estacion) & (df_data['FECHA']==fecha)]
            df_final_estacion = pd.concat([df_final_estacion, old_row])
            
            
    df_final = pd.concat([df_final, df_final_estacion])
    
    t_end_estacion = time.time()
    print('Fechas vacías:' + str(i))
    print('Tiempo: '+str(t_end_estacion-t_ini_estacion))
    i=0
    

t_end = time.time()
print (t_end - t_ini)

Estacion: 1
Fechas vacías:5048
Tiempo: 95.7553608417511
Estacion: 2
Fechas vacías:9374
Tiempo: 94.10419821739197
Estacion: 3
Fechas vacías:4498
Tiempo: 93.37680792808533
Estacion: 4
Fechas vacías:7270
Tiempo: 87.30157709121704
Estacion: 5
Fechas vacías:6080
Tiempo: 85.58908915519714
Estacion: 6
Fechas vacías:3550
Tiempo: 83.86037683486938
Estacion: 7
Fechas vacías:5579
Tiempo: 81.69315648078918
Estacion: 8
Fechas vacías:4590
Tiempo: 88.40373539924622
Estacion: 9
Fechas vacías:4355
Tiempo: 83.67577958106995
Estacion: 10
Fechas vacías:5669
Tiempo: 82.55328845977783
Estacion: 11
Fechas vacías:8419
Tiempo: 79.67317605018616
Estacion: 12
Fechas vacías:5325
Tiempo: 84.11241388320923
Estacion: 13
Fechas vacías:3339
Tiempo: 84.51745772361755
Estacion: 14
Fechas vacías:5737
Tiempo: 81.53916621208191
Estacion: 15
Fechas vacías:7629
Tiempo: 79.52067613601685
Estacion: 16
Fechas vacías:8050
Tiempo: 80.77215886116028
Estacion: 17
Fechas vacías:4766
Tiempo: 82.94149422645569
Estacion: 18
Fechas vací

Fechas vacías:5574
Tiempo: 80.46632599830627
Estacion: 146
Fechas vacías:9309
Tiempo: 77.6015899181366
Estacion: 147
Fechas vacías:10564
Tiempo: 77.32598423957825
Estacion: 148
Fechas vacías:7947
Tiempo: 80.06976103782654
Estacion: 149
Fechas vacías:3338
Tiempo: 82.37260127067566
Estacion: 150
Fechas vacías:10036
Tiempo: 77.3526041507721
Estacion: 151
Fechas vacías:10629
Tiempo: 77.17641258239746
Estacion: 152
Fechas vacías:9874
Tiempo: 77.47086811065674
Estacion: 153
Fechas vacías:6830
Tiempo: 79.86127233505249
Estacion: 154
Fechas vacías:6337
Tiempo: 79.42850255966187
Estacion: 155
Fechas vacías:6233
Tiempo: 80.00905156135559
Estacion: 156
Fechas vacías:6165
Tiempo: 80.10784935951233
Estacion: 157
Fechas vacías:4919
Tiempo: 80.76634240150452
Estacion: 158
Fechas vacías:8584
Tiempo: 78.58873724937439
Estacion: 159
Fechas vacías:9600
Tiempo: 77.60290288925171
Estacion: 160
Fechas vacías:4342
Tiempo: 81.33592343330383
Estacion: 161
Fechas vacías:3881
Tiempo: 81.94564867019653
Estacion: 

In [12]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3261120 entries, 0 to 2912137
Data columns (total 25 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ESTACION             object        
 1   ANIO                 object        
 2   MES                  object        
 3   DIA                  object        
 4   HORA                 object        
 5   FECHA                datetime64[ns]
 6   DIA_SEMANA           object        
 7   AM_PM                object        
 8   TEMPORADA            object        
 9   TEMPORADA_NUM        object        
 10  Es_Festivo           object        
 11  Es_FinSemana         object        
 12  TEMPERATURA          float64       
 13  VIENTO               float64       
 14  PRESION              float64       
 15  HUMEDAD              float64       
 16  PRECIPITACION_1h     float64       
 17  PRECIPITACION_3h     float64       
 18  DESC_TIEMPO          object        
 19  DESC_TIEMPO_detalle  

In [13]:
df_final['CLUSTER_FullCols'] = df_final['CLUSTER_FullCols'].astype(np.int64)
df_final['CLUSTER_soloGeo5'] = df_final['CLUSTER_soloGeo5'].astype(np.int64)
df_final['CLUSTER_soloGeo7'] = df_final['CLUSTER_soloGeo7'].astype(np.int64)
df_final['CLUSTER_soloDemanda'] = df_final['CLUSTER_soloDemanda'].astype(np.int64)
df_final['TEMPORADA_NUM'] = df_final['TEMPORADA_NUM'].astype(np.int64)
df_final['DEMANDA'] = df_final['DEMANDA'].astype(np.int64)

df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3261120 entries, 0 to 2912137
Data columns (total 25 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ESTACION             object        
 1   ANIO                 object        
 2   MES                  object        
 3   DIA                  object        
 4   HORA                 object        
 5   FECHA                datetime64[ns]
 6   DIA_SEMANA           object        
 7   AM_PM                object        
 8   TEMPORADA            object        
 9   TEMPORADA_NUM        int64         
 10  Es_Festivo           object        
 11  Es_FinSemana         object        
 12  TEMPERATURA          float64       
 13  VIENTO               float64       
 14  PRESION              float64       
 15  HUMEDAD              float64       
 16  PRECIPITACION_1h     float64       
 17  PRECIPITACION_3h     float64       
 18  DESC_TIEMPO          object        
 19  DESC_TIEMPO_detalle  

In [14]:
df_final

,ESTACION,ANIO,MES,DIA,HORA,FECHA,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,...,HUMEDAD,PRECIPITACION_1h,PRECIPITACION_3h,DESC_TIEMPO,DESC_TIEMPO_detalle,CLUSTER_FullCols,CLUSTER_soloGeo5,CLUSTER_soloGeo7,CLUSTER_soloDemanda,DEMANDA
0,1,2018,1,1,0,2018-01-01 00:00:00,2,AM,INVIERNO,1,...,56.0,0.0,0.0,Clear,sky is clear,4,4,1,0,0
0,1,2018,1,1,1,2018-01-01 01:00:00,2,AM,INVIERNO,1,...,55.0,0.0,0.0,Clear,sky is clear,4,4,1,0,0
0,1,2018,1,1,2,2018-01-01 02:00:00,2,AM,INVIERNO,1,...,60.0,0.0,0.0,Clear,sky is clear,4,4,1,0,0
0,1,2018,1,1,3,2018-01-01 03:00:00,2,AM,INVIERNO,1,...,59.0,0.0,0.0,Clear,sky is clear,4,4,1,0,0
0,1,2018,1,1,4,2018-01-01 04:00:00,2,AM,INVIERNO,1,...,57.0,0.0,0.0,Clear,sky is clear,4,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911680,175,2020,2,29,19,2020-02-29 19:00:00,7,PM,INVIERNO,1,...,60.0,0.0,0.0,Clouds,few clouds,0,0,1,2,8
2911809,175,2020,2,29,20,2020-02-29 20:00:00,7,PM,INVIERNO,1,...,61.0,0.0,0.0,Clouds,few clouds,0,0,1,2,3
2911921,175,2020,2,29,21,2020-02-29 21:00:00,7,PM,INVIERNO,1,...,67.0,0.0,0.0,Clouds,few clouds,0,0,1,2,1
2912014,175,2020,2,29,22,2020-02-29 22:00:00,7,PM,INVIERNO,1,...,69.0,0.0,0.0,Clouds,few clouds,0,0,1,2,2


In [15]:
df_final[df_final['DEMANDA']==0]

,ESTACION,ANIO,MES,DIA,HORA,FECHA,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,...,HUMEDAD,PRECIPITACION_1h,PRECIPITACION_3h,DESC_TIEMPO,DESC_TIEMPO_detalle,CLUSTER_FullCols,CLUSTER_soloGeo5,CLUSTER_soloGeo7,CLUSTER_soloDemanda,DEMANDA
0,1,2018,1,1,0,2018-01-01 00:00:00,2,AM,INVIERNO,1,...,56.0,0.0,0.0,Clear,sky is clear,4,4,1,0,0
0,1,2018,1,1,1,2018-01-01 01:00:00,2,AM,INVIERNO,1,...,55.0,0.0,0.0,Clear,sky is clear,4,4,1,0,0
0,1,2018,1,1,2,2018-01-01 02:00:00,2,AM,INVIERNO,1,...,60.0,0.0,0.0,Clear,sky is clear,4,4,1,0,0
0,1,2018,1,1,3,2018-01-01 03:00:00,2,AM,INVIERNO,1,...,59.0,0.0,0.0,Clear,sky is clear,4,4,1,0,0
0,1,2018,1,1,4,2018-01-01 04:00:00,2,AM,INVIERNO,1,...,57.0,0.0,0.0,Clear,sky is clear,4,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,175,2020,2,27,3,2020-02-27 03:00:00,5,AM,INVIERNO,1,...,66.0,0.0,0.0,Clouds,few clouds,0,0,1,2,0
0,175,2020,2,28,1,2020-02-28 01:00:00,6,AM,INVIERNO,1,...,54.0,0.0,0.0,Clear,sky is clear,0,0,1,2,0
0,175,2020,2,28,2,2020-02-28 02:00:00,6,AM,INVIERNO,1,...,57.0,0.0,0.0,Clear,sky is clear,0,0,1,2,0
0,175,2020,2,28,3,2020-02-28 03:00:00,6,AM,INVIERNO,1,...,60.0,0.0,0.0,Clear,sky is clear,0,0,1,2,0


In [18]:
df_final.groupby('CLUSTER_soloGeo5').count()

,ESTACION,ANIO,MES,DIA,HORA,FECHA,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,...,PRESION,HUMEDAD,PRECIPITACION_1h,PRECIPITACION_3h,DESC_TIEMPO,DESC_TIEMPO_detalle,CLUSTER_FullCols,CLUSTER_soloGeo7,CLUSTER_soloDemanda,DEMANDA
CLUSTER_soloGeo5,,,,,,,,,,,,,,,,,,,,,
0,910080,910080,910080,910080,910080,910080,910080,910080,910080,910080,...,910080,910080,910080,910080,910080,910080,910080,910080,910080,910080
1,530880,530880,530880,530880,530880,530880,530880,530880,530880,530880,...,530880,530880,530880,530880,530880,530880,530880,530880,530880,530880
2,549840,549840,549840,549840,549840,549840,549840,549840,549840,549840,...,549840,549840,549840,549840,549840,549840,549840,549840,549840,549840
3,417120,417120,417120,417120,417120,417120,417120,417120,417120,417120,...,417120,417120,417120,417120,417120,417120,417120,417120,417120,417120
4,853200,853200,853200,853200,853200,853200,853200,853200,853200,853200,...,853200,853200,853200,853200,853200,853200,853200,853200,853200,853200


### Validación de resultados en base a suma de DEMANDA

In [19]:
pd.set_option('display.max_rows', None)

In [20]:
# DF Original
df_tracks.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count'))
#df_tracks.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count')).to_clipboard()

,DEMANDA,CUENTA
ESTACION,,
1,90192,19681
2,42031,14344
3,69541,19659
4,48318,16108
5,44461,17600
6,91139,21066
7,53032,18130
8,61062,19377
9,103651,20090


In [21]:
# DF Corregido
df_final.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count'))
#df_final.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count')).to_clipboard()

,DEMANDA,CUENTA
ESTACION,,
1,63706,18960
2,28020,18960
3,52275,18960
4,35135,18960
5,32312,18960
6,66034,18960
7,39552,18960
8,45164,18960
9,75799,18960


In [22]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3261120 entries, 0 to 2912137
Data columns (total 25 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ESTACION             object        
 1   ANIO                 object        
 2   MES                  object        
 3   DIA                  object        
 4   HORA                 object        
 5   FECHA                datetime64[ns]
 6   DIA_SEMANA           object        
 7   AM_PM                object        
 8   TEMPORADA            object        
 9   TEMPORADA_NUM        int64         
 10  Es_Festivo           object        
 11  Es_FinSemana         object        
 12  TEMPERATURA          float64       
 13  VIENTO               float64       
 14  PRESION              float64       
 15  HUMEDAD              float64       
 16  PRECIPITACION_1h     float64       
 17  PRECIPITACION_3h     float64       
 18  DESC_TIEMPO          object        
 19  DESC_TIEMPO_detalle  

In [23]:
pd.set_option('display.max_rows', 10)

### Export df_final a CSV

In [24]:
df_final.to_csv('../../data/DataFrame_Final_Cierre_Cluster_FullFecha.csv', index=False)
#df_final.to_csv('../../data/test.csv', index=False)

In [ ]:
test=df_final.groupby(by='TEMPORADA').count()
test

In [ ]:
test=df_final.groupby(by='ANIO').count()
test

In [ ]:
df_final.groupby('ANIO').agg(DEMANDA=('DEMANDA', 'sum'))

In [ ]:
DF2 = pd.read_csv('../../data/test.csv', parse_dates=['FECHA'])
DF2.info()

In [ ]:
import pandas as pd
df_tracks = pd.read_csv('../../data/DataFrame_Final_Cierre_Cluster_FullFecha.csv', parse_dates=['FECHA'])

In [ ]:
df_tracks.info()